In [66]:
import datetime
import json

import boto3
import requests

## Part 1.

**1.1 Fetch the top viewed pages for a specific date** 

In [67]:
DATE_PARAM = "2025-11-29"
date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

In [68]:
# Try different dates to see how the data changes
USERNAME = "ambrus"

date = datetime.datetime.strptime(DATE_PARAM, "%Y-%m-%d")

# Construct the API URL
url = f"https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/{date.year}/{date.month}/{date.day}"
print(f"Requesting REST API URL: {url}")

# Make the API request
wiki_server_response = requests.get(url, headers={"User-Agent": "curl/7.68.0"})
wiki_response_status = wiki_server_response.status_code
wiki_response_body = wiki_server_response.text

print(f"Wikipedia REST API Response body: {wiki_response_body[:500]}...")
print(f"Wikipedia REST API Response Code: {wiki_response_status}")

# Validate response
if wiki_response_status != 200:
    raise Exception(f"Received non-OK status code from Wiki Server: {wiki_response_status}")
print(f"Successfully retrieved Wikipedia data, content-length: {len(wiki_response_body)}")

Requesting REST API URL: https://wikimedia.org/api/rest_v1/metrics/pageviews/top/en.wikipedia.org/all-access/2025/11/29
Wikipedia REST API Response body: {"items":[{"project":"en.wikipedia","access":"all-access","year":"2025","month":"11","day":"29","articles":[{"article":"Main_Page","views":6105736,"rank":1},{"article":"Special:Search","views":790155,"rank":2},{"article":"Stranger_Things_season_5","views":410638,"rank":3},{"article":"Stranger_Things","views":275039,"rank":4},{"article":"Google_Chrome","views":271946,"rank":5},{"article":"Wikipedia:Featured_pictures","views":243015,"rank":6},{"article":"Tom_Stoppard","views":240997,"rank":7},{"ar...
Wikipedia REST API Response Code: 200
Successfully retrieved Wikipedia data, content-length: 55893


**1.2 Transform the response to JSON Lines format with these fields:**


In [69]:
# Parse the API response and extract top edits
wiki_response_parsed = wiki_server_response.json()
top_views = wiki_response_parsed["items"][0]["articles"]

# Transform to JSON Lines format
current_time = datetime.datetime.now(datetime.timezone.utc)
json_lines = ""
for page in top_views:
    record = {
        "title": page["article"],
        "views": page["views"],
        "rank": page["rank"],
        "date": date.strftime("%Y-%m-%d"),
        "retrieved_at": current_time.replace(tzinfo=None).isoformat(),
    }
    json_lines += json.dumps(record) + "\n"

print(f"Transformed {len(top_views)} records to JSON Lines")
print(f"First few lines:\n{json_lines[:500]}...")

Transformed 999 records to JSON Lines
First few lines:
{"title": "Main_Page", "views": 6105736, "rank": 1, "date": "2025-11-29", "retrieved_at": "2025-12-10T20:15:01.806820"}
{"title": "Special:Search", "views": 790155, "rank": 2, "date": "2025-11-29", "retrieved_at": "2025-12-10T20:15:01.806820"}
{"title": "Stranger_Things_season_5", "views": 410638, "rank": 3, "date": "2025-11-29", "retrieved_at": "2025-12-10T20:15:01.806820"}
{"title": "Stranger_Things", "views": 275039, "rank": 4, "date": "2025-11-29", "retrieved_at": "2025-12-10T20:15:01.806820...


**1.3 Upload to S3 at raw-views/raw-views-YYYY-MM-DD.json**

In [70]:
s3 = boto3.client(
    "s3",
    region_name="eu-west-1",  # Ireland region
)
default_region = "eu-west-1"

In [71]:
S3_WIKI_BUCKET = "ambrus-wikidata"
file_name = f"raw-views/raw-views-{DATE_PARAM}.json"
s3.put_object(Bucket=S3_WIKI_BUCKET, Body=json_lines, Key=file_name)

{'ResponseMetadata': {'RequestId': '940RVCPGXY20QAER',
  'HostId': 'hvtY0Y/cquo7++gFZCO/MEuxiNL+vplSm7e+SVvIAPhVAmxeshkn8oMqSyQftldGdh29Zs/+rmE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'hvtY0Y/cquo7++gFZCO/MEuxiNL+vplSm7e+SVvIAPhVAmxeshkn8oMqSyQftldGdh29Zs/+rmE=',
   'x-amz-request-id': '940RVCPGXY20QAER',
   'date': 'Wed, 10 Dec 2025 20:15:03 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"4b0bf3853fa6866f6c946a53bdcef2d6"',
   'x-amz-checksum-crc32': '7M1BLA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4b0bf3853fa6866f6c946a53bdcef2d6"',
 'ChecksumCRC32': '7M1BLA==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

## Part 2: Create Athena Tables

[4_raw_views.sql](./4_raw_views.sql)

[5_views_view.sql](5_views_view.sql)